In [0]:
import nltk
nltk.download('punkt')
import math
import re
import numpy as np
from nltk.util import ngrams

import gensim
from keras.callbacks import LambdaCallback
from keras.layers.recurrent import LSTM
from keras.layers.embeddings import Embedding
from keras.layers import Dense, Activation, SimpleRNN, Dropout
from keras.models import Sequential
from keras.utils.data_utils import get_file

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Using TensorFlow backend.


In [0]:
!wget https://raw.githubusercontent.com/ryanmcdermott/trump-speeches/master/speeches.txt

--2019-10-06 19:36:47--  https://raw.githubusercontent.com/ryanmcdermott/trump-speeches/master/speeches.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 924745 (903K) [text/plain]
Saving to: ‘speeches.txt’

speeches.txt        100%[===================>] 903.07K  --.-KB/s    in 0.02s   

2019-10-06 19:36:47 (38.3 MB/s) - ‘speeches.txt’ saved [924745/924745]

--2019-10-06 19:36:50--  https://raw.githubusercontent.com/ryanmcdermott/trump-speeches/master/speeches.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 924745 (903K) [text/plain]


**Text Preprocessing**

In [0]:
with open('speeches.txt') as f:
  text = f.read()
lower_text=nltk.sent_tokenize(text.lower())
sentences=[]
for line in lower_text:
  if re.search(r'[a-z0-9]', line):
    l=re.sub(r'[^a-z0-9]'," ", line)
  sentences.append(l)

**Train Test Split**

In [0]:
train = sentences[:math.ceil(0.8*len(sentences))]
test = sentences[math.ceil(0.8*len(sentences)):]

**Classic Modelling N grams**

In [0]:
def Ngram_model(n, data):
  ngram_list=[]
  for sentence in data:
    sentence=nltk.word_tokenize(sentence)
    padded_sent = list(['<s>']+sentence+['</s>'])
    ngram_list.extend(list(ngrams(padded_sent, n=n)))
  return ngram_list

In [0]:
def Freq_Dist(ngram_list):
  freq=nltk.FreqDist(ngram_list)
  freq_dist={}
  for key in freq:
    freq_dist[key]=freq[key]
  return freq_dist

In [0]:
def MLE_dict(n, data):
  mle_dict={}
  l1=Ngram_model(n, data)
  f1=Freq_Dist(l1)
  if(n!=1):
    l2=Ngram_model(n-1, data)
    f2=Freq_Dist(l2)
    for key in f1:
      x=(' '.join(key))
      y=f1[key]/f2[key[:-1]]
      mle_dict[x]=y
  else:
    for key in f1:
      x=(' '.join(key))
      y=f1[key]/len(l1)
      mle_dict[x]=y
  return mle_dict

In [0]:
def Generator(mle_dict):
  sentence=[]
  pvalues=np.array([mle_dict[key] for key in mle_dict])
  pvalues=pvalues/pvalues.sum()
  while(True):
    poss=list(np.random.multinomial(20, pvalues))
    start=list(mle_dict.keys())[poss.index(max(poss))]
    if '<s>' in start:
      break
    else:
      pass
  sentence=sentence+start.split(' ')
  while(True):
    poss=list(np.random.multinomial(20, pvalues))
    move=list(mle_dict.keys())[poss.index(max(poss))]
    if('</s>' not in move and '<s>' not in move):
      sentence=sentence+move.split(' ')
      pass
    elif('</s>' in move):
      sentence=sentence+move.split(' ')
      break
    
  return ' '.join(sentence)

In [0]:
def Perplexity(n, mle_dict, test):
  ngram_test=Ngram_model(n, test)
  perplexity=0
  N=len(ngram_test)
  m=min(list(mle_dict.values()))
  for key in ngram_test:
    if key in mle_dict:
      perplexity=perplexity-(math.log(mle_dict(key))/N)
    else:
      perplexity=perplexity-((math.log(m))/N)
  perplexity=math.exp(perplexity)
  return perplexity

**Perplexity of N gram Models**

In [0]:
for n in range(1, 5):
  mle=MLE_dict(n, train)
  p=Perplexity(n, mle, test)
  print("Perplexity of "+str(n)+"-gram : "+str(p))

Perplexity of 1-gram : 167970.00000018327
Perplexity of 2-gram : 13120.999999960633
Perplexity of 3-gram : 1893.0000000043613
Perplexity of 4-gram : 719.0000000017544


**Observation : Perplexity of unigram > bigram > trigram > quadgram**

**Random Text Generation of N gram Models**

In [0]:
for n in range(1, 5):
  mle=MLE_dict(n, train)
  print("Generation text : "+str(n)+"-gram")
  print('')
  for i in range(5):
    print(Generator(mle))
  print('')

Generation text : 1-gram

<s> </s>
<s> </s>
<s> a the to people </s>
<s> </s>
<s> </s>

Generation text : 2-gram

<s> i years </s>
<s> i courts because with the much </s>
<s> what executive order islamic terrorism void much repeal and consulate and rusted and liked him crooks </s>
<s> i lied about also need kicks in pennsylvania avenue action </s>
<s> it wants to people we watch this japanese imperialists fall </s>

Generation text : 3-gram

<s> civil war deficit that we me folks </s>
<s> again the amounts of money than donald trump dollar is getting truly wealthy again re fighting here lot to be conservative and i know somebody is maybe two maybe television this morning cut social security rebuild quickly our ago pull out press he doesn weaken america he see more than can save social dollars were lost thinking of running bridge on every moneys lost </s>
<s> horribly wasted i file my biden again criticized proposed a 2017 c this plan doing an absolutely economic assault on way especial

**Observation : Readability of quadgram > trigram > bigram > unigram**

**Neural Language Modelling : LSTM and RNN**

In [0]:
def Neural_training(data, model_name):
  
  max_length=20
  sent_data=[['<s>']+line.split()[:max_length]+['</s>'] for line in data]
  
  word_model = gensim.models.Word2Vec(sent_data, size=10, min_count=1, iter=100)
  trained_vocabs = word_model.wv.vocab
  trained_weights = np.array(word_model.wv.vectors)
  vocab_size = word_model.wv.vectors.shape[0]
  embedding_size = word_model.wv.vectors.shape[1]
  
  model = Sequential()
  model.add(Embedding(input_dim=vocab_size, output_dim=embedding_size, weights=[trained_weights]))
  if(model_name=='LSTM'):
    model.add(LSTM(units=embedding_size))
  if(model_name=='RNN'):
    model.add(SimpleRNN(units=embedding_size))
  model.add(Dropout(0.1))
  model.add(Dense(units=vocab_size))
  model.add(Activation('softmax'))
  model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')
  
  corpus=[]
  for i, line in enumerate(sent_data):
    for j in range(2, len(line)):
      corpus.append(line[:j])
  
  train_X = np.zeros([len(corpus), max_length+2], dtype=np.int32)
  train_y = np.zeros([len(corpus)], dtype=np.int32)
  for i, line in enumerate(corpus):
    for j, word in enumerate(line[:-1]):
      train_X[i, j] = word_model.wv.vocab[word].index
    train_y[i]=word_model.wv.vocab[line[-1]].index
  model.fit(train_X, train_y, batch_size=128, epochs=5)
  
  return (word_model, model)

In [0]:
def Neural_generator(word_model, model):
  x=['<s>']
  i=0
  while(i<=10):
    i=i+1
    windex_x=[word_model.wv.vocab[word].index for word in x]
    preds = model.predict(x=np.array([windex_x]))
    preds=preds/preds.sum()
    p = np.random.multinomial(1, preds[0], 1)
    idx = np.argmax(p)
    w=word_model.wv.index2word[idx]
    if(w=='</s>'):
      x.append(w)
      break
    else:
      x.append(w)
      pass
  return(' '.join(x))

In [0]:
def Neural_perplexity(model, test):
  
  max_length=20
  sent_data=[['<s>']+line.split()[:max_length]+['</s>'] for line in test]
  
  word_model = gensim.models.Word2Vec(sent_data, size=10, min_count=1, iter=100)
  trained_vocabs = word_model.wv.vocab
  trained_weights = np.array(word_model.wv.vectors)
  vocab_size = word_model.wv.vectors.shape[0]
  embedding_size = word_model.wv.vectors.shape[1]
  
  corpus=[]
  for i, line in enumerate(sent_data):
    for j in range(2, len(line)):
      corpus.append(line[:j])
        
  N=len(corpus)
  perplexity=0
  for i, line in enumerate(corpus):
    x=line[:-1]
    y=line[-1]
    windex_x=[word_model.wv.vocab[word].index for word in x]
    preds=model.predict(x=np.array([windex_x]))
    idx=word_model.wv.vocab[y].index
    prob=preds[0][idx]
    perplexity=perplexity-(math.log(prob)/N)
  perplexity=math.exp(perplexity)
  
  return perplexity

**Random Text Generation of Neural Models using start words**

In [0]:
rnn_word_model, rnn_model =Neural_training(train, 'RNN')

Epoch 1/5
127009/127009 [==============================] - 28s 220us/step - loss: 6.3952
Epoch 2/5
127009/127009 [==============================] - 27s 213us/step - loss: 5.9917
Epoch 3/5
127009/127009 [==============================] - 27s 212us/step - loss: 5.9867
Epoch 4/5
127009/127009 [==============================] - 27s 211us/step - loss: 5.9834
Epoch 5/5
127009/127009 [==============================] - 25s 199us/step - loss: 5.9843


In [0]:
lstm_word_model, lstm_model =Neural_training(train, 'LSTM')

Epoch 1/5
127009/127009 [==============================] - 41s 321us/step - loss: 6.3455
Epoch 2/5
127009/127009 [==============================] - 37s 288us/step - loss: 5.9558
Epoch 3/5
127009/127009 [==============================] - 38s 299us/step - loss: 5.8646
Epoch 4/5
127009/127009 [==============================] - 40s 312us/step - loss: 5.7888
Epoch 5/5
127009/127009 [==============================] - 40s 315us/step - loss: 5.7237


In [0]:
print("Generation text : RNN")
for i in range(5):
  print(Neural_generator(rnn_word_model, rnn_model))

Generation text : RNN
<s> subsidies veterans grid veteran screaming world non donors arkansas fraud re
<s> m bring poker job laws gates deductible scum cheerleader hostile opinion
<s> year fellas babies have doubt fellas more equipment while making artificially
<s> t saying have follows we being sought local ozone history reinvigorate
<s> proud emails this we raised speaking of her practiced nevada they


In [0]:
print("Generation text : LSTM")
for i in range(5):
  print(Neural_generator(lstm_word_model, lstm_model))

Generation text : LSTM
<s> seeing look s from insist the space going gen nice settled
<s> hope require fantastic day year exercise gorgeous phoenix starts fantasy elected
<s> candidates 5 listen sometimes yes in peanuts of tur joining victory
<s> cities terrific famous made 60 truck friend 24 certainly instincts indie
<s> crowds cities 7 schuster poll bullets anger highest land monstrous kristol


**Observation : Readabilty of LSTM > RNN**

**Perplexity of Neural Models**

In [0]:
p_rnn=Neural_perplexity(rnn_model, test)
p_lstm=Neural_perplexity(lstm_model, test)

In [0]:
print("Perplexity RNN : "+str(p_rnn))
print("Perplexity LSTM : "+str(p_lstm))

Perplexity RNN : 927.2233875816244
Perplexity LSTM : 762.466345402934


**Observation : Perplexity of RNN > LSTM**

**Neural Models perform better than Classical Models. Because of the recurrence property of these RNN Models. It can be shown through the low perplexities of neural models compared to classic models based on the fact that model is better when perplexity is low.**